# Previsão com o Modelo de OCREste notebook demonstra como carregar o modelo de CNN treinado e utilizá-lo para prever a classe de uma imagem de caractere. As etapas incluem:- **Carregamento do Modelo e Classes**: Carrega o arquivo do modelo `.keras` e o mapeamento de classes `class_names.json`.- **Funções Auxiliares**: Define funções para converter entre o formato `U+XXXX` e índices de classe.- **Seleção e Preparação da Imagem**: Escolhe uma imagem do conjunto de teste, a carrega e a pré-processa para que seja compatível com a entrada do modelo.- **Execução da Previsão**: Alimenta a imagem no modelo e obtém as probabilidades de cada classe.- **Análise dos Resultados**: Exibe a classe prevista, a confiança da previsão e as 5 classes mais prováveis.

## Importação de BibliotecasImporta as bibliotecas necessárias, incluindo TensorFlow para carregar o modelo e realizar previsões, Matplotlib para visualização e `os` e `pathlib` para manipulação de arquivos.

In [63]:
import tensorflow as tffrom tensorflow.keras import layers, modelsimport matplotlib.pyplot as pltimport numpy as npimport osfrom pathlib import Path

## Localização da Raiz do ProjetoA função `find_project_root` localiza o diretório raiz para resolver os caminhos dos arquivos de modelo, classes e dataset de forma robusta.

In [64]:
def find_project_root(markers=(pyproject.toml, .git)) -> Path:    '''    Descobre a raiz do projeto subindo diretórios até encontrar um marcador.    Parâmetros:        markers (tuple): Nomes de arquivos/pastas que indicam a raiz.    Retorna:        pathlib.Path: Aponta para a raiz do projeto.    '''    p = Path.cwd().resolve()    for parent in [p, *p.parents]:        if any((parent / m).exists() for m in markers):            return parent    return pROOT = find_project_root()

## Configuração de CaminhosDefine os caminhos para o modelo salvo, o arquivo de nomes de classes e o diretório do dataset.

In [65]:
# MODEL_SAVE_PATH = 'modelo_ocr_simbolos.keras'DATA_DIR = ROOT / dataMODEL_SAVE_PATH = DATA_DIR / processed / modelo_ocr_simbolos.kerasCLASS_NAMES_PATH = DATA_DIR / raw / class_names.jsonDATASET_DIR = DATA_DIR / raw / dataset

## Verificação de Consistência das ClassesEsta célula de verificação garante que a ordem das classes no `class_names.json` corresponde à ordem alfabética dos diretórios de treinamento, que é a ordem usada pelo TensorFlow. Isso é essencial para que as previsões do modelo sejam mapeadas para os rótulos corretos.

In [66]:
# Verificar se class_names.json está na ordem correta (alfabética)import jsonwith open(CLASS_NAMES_PATH, 'r', encoding='utf-8') as f:    loaded_classes = json.load(f)# class_names do dataset de treino (ordem usada pelo modelo)train_dir = os.path.join(DATASET_DIR, 'train')train_classes_sorted = sorted(os.listdir(train_dir))print(Verificando consistência de class_names...)if loaded_classes == train_classes_sorted:    print(✓ class_names.json está correto e consistente com o treinamento)else:    print(✗ ERRO: class_names.json difere da ordem do treinamento!)    print(f  Esperado (train): {train_classes_sorted[:5]}...)    print(f  Carregado (json): {loaded_classes[:5]}...)    print(RECOMENDAÇÃO: Regere class_names.json com:)    print(  import json)    print(  with open(CLASS_NAMES_PATH, 'w', encoding='utf-8') as f:)    print(      json.dump(sorted(os.listdir('dataset/train')), f, ensure_ascii=False))

## Carregamento dos Nomes das ClassesCarrega os nomes das classes do arquivo JSON para uma lista, que será usada para mapear os índices de saída do modelo para os rótulos de caractere correspondentes.

In [67]:
# Carrega os nomes das classes a partir do arquivo 'class_names.json'import jsonwith open(CLASS_NAMES_PATH, 'r', encoding='utf-8') as f:    class_names = json.load(f)print(f"Nomes das classes carregados: {len(class_names)} classes.")for idx, name in enumerate(class_names):    print(idx, name)

## Funções Utilitárias para CodepointsConjunto de funções para conversão entre a notação `U+XXXX` e valores inteiros de codepoint ou índices de lista. Isso facilita a seleção de imagens de teste e a interpretação dos resultados.

In [72]:
import redef parse_uplus(s: str) -> int:    '''Converte uma string no formato 'U+XXXX' para um inteiro.    Args:        s (str): A string a ser convertida (ex: 'U+0044').    Returns:        int: O valor inteiro do codepoint.    '''    m = re.fullmatch(r'(?:U\\+|0x)?([0-9A-Fa-f]+)', s.strip())    if not m:        raise ValueError(fFormato inválido: {s})    return int(m.group(1), 16)def uplus_to_hex(s: str) -> str:    '''Converte uma string 'U+XXXX' para sua representação hexadecimal '0xXXXX'.    Args:        s (str): A string de entrada.    Returns:        str: A string formatada em hexadecimal.    '''    cp = parse_uplus(s)    return f0x{cp:04X}def uplus_to_index_int(s: str, base: int = 0x0020) -> int:    '''Converte 'U+XXXX' para um índice de lista, subtraindo uma base.    Args:        s (str): A string de entrada.        base (int): O valor base a ser subtraído.    Returns:        int: O índice resultante.    '''    cp = parse_uplus(s)    idx = cp - base    if idx < 0:        raise ValueError(fÍndice negativo: codepoint 0x{cp:04X} antes da base 0x{base:04X})    return idxdef uplus_to_index_hex(s: str, base: int = 0x0020) -> str:    '''Converte 'U+XXXX' para um índice hexadecimal, subtraindo uma base.    Args:        s (str): A string de entrada.        base (int): O valor base a ser subtraído.    Returns:        str: O índice hexadecimal resultante.    '''    idx = uplus_to_index_int(s, base=base)    return f0x{idx:04X}# Exemplos:# uplus_to_hex('U+0044')               -> '0x0044'# uplus_to_index_hex('U+0044', 0x0020) -> '0x0024'  (inclui espaço)# uplus_to_index_hex('U+0044', 0x0021) -> '0x0023'  (ignora espaço, corresponde ao seu exemplo)

## Configuração da Imagem de TesteDefine qual caractere (`SYMBOL_UNICODE`) será usado para o teste de previsão e calcula seu índice correspondente na lista de classes.

In [ ]:
# DATASET_DIR = 'dataset'IMG_HEIGHT = 64IMG_WIDTH = 64SYMBOL_UNICODE = 'U+0044'  # Exemplo: 'U+0044' para 'D'SYMBOL_INDEX = uplus_to_index_int(SYMBOL_UNICODE, base=0x0021)

## Carregamento do Modelo e Visualização de AmostrasCarrega o modelo treinado do arquivo `.keras`. Em seguida, exibe uma grade de imagens de amostra da classe de teste selecionada para fornecer um contexto visual do que está sendo previsto.

In [74]:
'''Teste rápido de previsão'''# Carrega o modelo salvomodel = tf.keras.models.load_model(MODEL_SAVE_PATH)print(Modelo carregado para teste de previsão.)# Carrega uma imagem de testetest_image_path = os.path.join(DATASET_DIR, 'test', class_names[SYMBOL_INDEX])# Exibe algumas imagens de testeplt.figure(figsize=(10, 10))for i in range(9):    ax = plt.subplot(3, 3, i + 1)    img_path = os.path.join(test_image_path, os.listdir(test_image_path)[i])    img = tf.keras.preprocessing.image.load_img(img_path, color_mode='grayscale', target_size=(IMG_HEIGHT, IMG_WIDTH))    plt.imshow(img, cmap='gray')    plt.axis(off)plt.show()

## Seleção e Exibição de uma Amostra EspecíficaSeleciona uma única imagem do conjunto de teste e a exibe, confirmando visualmente a imagem que será usada para a previsão.

In [75]:
# Selecionar e exibir imagem de testesample_idx = 20sample_image_path = os.path.join(test_image_path, os.listdir(test_image_path)[sample_idx])sample_image = tf.keras.preprocessing.image.load_img(    sample_image_path,    target_size=(IMG_HEIGHT, IMG_WIDTH),    color_mode='grayscale')plt.imshow(sample_image, cmap='gray')plt.title(fClasse real: {class_names[SYMBOL_INDEX]})plt.axis('off')plt.show()

## Preparação, Previsão e Análise do ResultadoA imagem de amostra é convertida em um array NumPy e formatada para corresponder ao formato de entrada do modelo (incluindo a dimensão do lote). O modelo então faz a previsão. Os resultados são analisados para encontrar a classe com a maior probabilidade, que é então mapeada de volta para o rótulo `U+XXXX` e exibida junto com a confiança e as 5 principais previsões.

In [76]:
# CORREÇÃO: Preparar imagem para previsão (SEM normalização manual)input_arr = tf.keras.preprocessing.image.img_to_array(sample_image)# garantir shape correto (H, W, 1)if input_arr.ndim == 2:    input_arr = np.expand_dims(input_arr, axis=-1)# adicionar dimensão do batch: (1, H, W, 1)input_arr = np.expand_dims(input_arr, axis=0)# IMPORTANTE: NÃO dividir por 255 aqui — o modelo já tem Rescaling(1./255) interno# Se você dividir aqui, os valores ficarão 0..1/255 (muito pequenos)predictions = model.predict(input_arr)predicted_idx = np.argmax(predictions)predicted_class = class_names[predicted_idx]confidence = predictions[0][predicted_idx]print(fClasse real (pasta): {class_names[43]})print(fClasse prevista: {predicted_class})print(fConfiança: {confidence:.4f})print(fTop-5 previsões:)top5_idx = np.argsort(predictions[0])[-5:][::-1]for i, idx in enumerate(top5_idx):    print(f  {i+1}. {class_names[idx]}: {predictions[0][idx]:.4f})